In [ ]:
import ollama
import requests as rq
from bs4 import BeautifulSoup as bs
import random
from telethon import TelegramClient, sync

# выбор модели
model_name = "llama3.2" 
print("Используемая модель:", model_name)


#____ФУНКЦИЯ SUMMARY С OLLAMA____
def summary_text(text_in):
    """
    Выделяет самую значимую информацию из текста.
    Возвращает summary - краткая сводка (1-2 предложения);
    список - bullets - несколько ключевых пунктов;
    keywords - ключевые слова.
    """
    # первичный запрос
    prompt = (
        "Верни СТРОГО JSON в таком формате: {\"summary\": <2–3 предложения>, \"bullets\": [3 пункта], \"keywords\": [5 слов]}."
        + text_in
    )
    # ответ трёх проходок с разным seed
    text_before = ""
    for _ in range(3):
        seed = random.randint(0, 100)
        resp_common = ollama.generate(
            model=model_name,
            prompt=prompt,
            options={"temperature": 0.3, "top_p": 0.8, "seed": seed, "num_predict": 300}
        )
        text = resp_common["response"].strip()
        text_before += text + "\n\n"    
    # обработка ответа
    prompt_clean = (
        "Из всех summary выдели только наиболее значимую информацию, оставь ≤ 35 слов"
        "Из всех bullets выдели только наиболее значимые, оставь до ≤ 12 слов" 
        "keywords — только существительные, без дубликатов"
        "Верни СТРОГО JSON в таком формате: {\"summary\": <2–3 предложения>, \"bullets\": [3 пункта], \"keywords\": [5 слов]}."
        + text_before
    )
    resp_clear = ollama.generate(
        model=model_name,
        prompt=prompt_clean,
        options={"temperature": 0.2, "top_p": 0.8, "seed": 8, "num_predict": 300}
    )
    text_after = resp_clear["response"].strip()
    return text_after


#____ПАРСИНГ ВЕБ-САЙТА (основная статья "Фонтанки")____

# User-Agent
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) '
           'Gecko/20100101 Firefox/88.0'}

# ссылка на главную страницу фонтанки
url_fontanka = 'https://www.fontanka.ru/'
# главная страница фонтанки
response_fontanka_main = rq.get(url=url_fontanka, headers=headers)
response_fontanka_main.raise_for_status()  # проверка успешности запроса
# объект BeautifulSoup главной страницы
soup_fontanka = bs(response_fontanka_main.text, 'html.parser')

# ссылка на основную статью на главной странице
main_art_url = (soup_fontanka.find('article', {'class':'wrap_4T9MT'})).find('a')['href']
# страница основной статьи
response_main_art = rq.get(url=main_art_url, headers=headers)
response_main_art.raise_for_status()  # проверка успешности запроса
# объект BeautifulSoup основной статьи
soup_main_art = bs(response_main_art.text, 'html.parser')

# сбор текста по структуре кода страницы фонтанки
text_elements = soup_main_art.select('div[class^="uiArticleBlockText"]')
main_art_text = ' '.join([elem.get_text(strip=True) for elem in text_elements])

def normalize_text(text):
    """Нормализует текст с проблемной кодировкой"""
    if isinstance(text, str):
        # Исправляем распространённые проблемы с кодировкой
        normalized = text.encode('latin-1').decode('utf-8', errors='ignore')
        return normalized
    return text
    
# декодировка текста основной статьи
main_art_text = normalize_text(main_art_text)


#____ПАРСИНГ VK (первые 10 постов со стены открытого сообщества)____

# переменные 
token_user = ''   # user_token
version = '5.199'
domain =  ''      # открытое сообщество

# обращение к api vk 
response = rq.get('https://api.vk.com/method/wall.get',
    params={'access_token': token_user,
            'v': version,
            'domain': domain,
            'count': 10,
            'filter': 'owner'})
# успешность запроса
if response.status_code == 200:
    data = response.json()
    # наличие ошибок в ответе VK
    if 'error' in data:
        print(f"Ошибка VK API: {data['error']['error_msg']}")
    else:
        posts = data['response']['items']
        # тексты всех постов
        post_texts = []
        for post in posts:
            text = post.get('text', '').strip()
            if text:  # только непустые тексты
                post_texts.append(text)
# объединение текста всех постов в одну строку
all_posts_text = " ".join(post_texts) 


#____ПАРСИНГ TG (первые 100 публикаций telegram-канала)____

# параметры обращения к API
api_id = ''           # api id
api_hash = ''         # api hash
channel_username = '' # username канала
async def main():
    async with TelegramClient('session_name', api_id, api_hash) as client:
        messages = await client.get_messages(channel_username, limit=100)
        post_tg_texts = []  # список для текстов
        for message in messages:
            if message.text:  # текст есть
                post_tg_texts.append(message.text)
        return post_tg_texts  # список
# тексты публикаций тг
tg_texts = await main()
# все тексты публикаций в одну строку
all_tg_texts = " ".join(tg_texts) 


#____ВЫЗОВ___
if __name__ == '__main__':
    text_font = summary_text(main_art_text)
    print(text_font)
    text_vk_group = summary_text(all_posts_text)
    print(text_vk_group)
    text_tg_channel = summary_text(all_tg_texts)
    print(text_tg_channel)

In [66]:
#____ПАРСИНГ TG (первые 100 публикаций telegram-канала)____

# API ID и API Hash
api_id = '30222534'
api_hash = 'd37177ed456e9fac88e613b077440214'
# username канала
channel_username = 'OutCinema'

async def main():
    async with TelegramClient('session_name', api_id, api_hash) as client:
        messages = await client.get_messages(channel_username, limit=100)
        post_tg_texts = []  # список для текстов
        for message in messages:
            if message.text:  # текст есть
                post_tg_texts.append(message.text)
                print(f"ID: {message.sender_id}")
                print(f"Текст: {message.text}")
                print("-" * 50)
        return post_tg_texts  # список
# тексты публикаций тг
tg_texts = await main()

ID: -1001430589779
Текст: **Предпремьерный показ «Человека-слона» Дэвида Линча в Доме Радио**

23 ноября [Дом Радио](https://t.me/domradio_online) — центр культуры, резиденция оркестра, хора musicAeterna и танцевальной труппы musicAeterna Dance — откроет свои двери для незабываемого кинематографического события. В его стенах, где витает дух академической музыки и таинства, мы представим **отреставрированную версию шедевра Дэвида Линча **__«Человек-слон».__

Мы приглашаем вас на уникальный сеанс, где хрупкая грань между прошлым и настоящим, между болью и красотой, будет ощущаться особенно сильно.

[| присоединиться к мероприятию |](https://outcinema.ru/theelephantman)
--------------------------------------------------
ID: -1001430589779
Текст: **Грим, изменивший Оскар: Как создавали одного из самых сложных персонажей в истории кино**

Создание грима для __«Человека-слона»__ Дэвида Линча стало настоящей драмой за кадром. Режиссёр, движимый своим видением,** хотел сам воссоздать облик Джо

In [52]:
#____ПАРСИНГ VK ПОДРОБНО (первые 10 постов со стены открытого сообщества)____

# переменные 
token_user = '7400270a7400270a7400270a75773cf3a6774007400270a1d160aae81a37df599eb5d2b'
version = '5.199'
domain =  'glagol_theatre'

# через api vk вызываем статистику постов
response = rq.get('https://api.vk.com/method/wall.get',
    params={'access_token': token_user,
            'v': version,
            'domain': domain,
            'count': 10,
            'filter': 'owner'})

# успешность запроса
if response.status_code == 200:
    data = response.json()
    
    # наличие ошибок в ответе VK
    if 'error' in data:
        print(f"Ошибка VK API: {data['error']['error_msg']}")
    else:
        posts = data['response']['items']
        
        # тексты всех постов
        post_texts = []
        for post in posts:
            text = post.get('text', '').strip()
            if text:  # только непустые тексты
                post_texts.append(text)
        
        # результаты
        print(f"Найдено постов: {len(posts)}")
        print(f"Постов с текстом: {len(post_texts)}")
        print("\n" + "="*50)
        
        for i, text in enumerate(post_texts, 1):
            print(f"Пост #{i}:")
            print(text)
            print("-" * 30)
            
else:
    print(f"Ошибка HTTP: {response.status_code}")

Найдено постов: 10
Постов с текстом: 9

Пост #1:
Уважаемые зрители!
Народный университетский театр «Глагол» [club61195360|СПбПУ] публикует свой дальнейший репертуар на осень:

25.10 19:00 – «Приблизительно – в сторону солнца» (16+) (ПРЕМЬЕРА)
02.11 (Вс) 19:00 – «Приблизительно – в сторону солнца» (16+) (ПРЕМЬЕРА)
08.11 18:00 – «Послезавтра – в Шамборе» (16+) 
15.11 18:00 – «Послезавтра – в Шамборе» (16+) 
22.11 18:00 – «Что угодно» (16+)
29.11 18:00 – «Что угодно» (16+)
06.12 19:00 – «Смешно в 2000-м году...» (16+)

В репертуаре возможны изменения – следите за объявлениями в группе!
Напоминаем о [https://vk.ru/topic-370748_47101903|правилах бронирования и посещения] нашего театра.

Запись на спектакли проходит в данной группе по субботам в 16 часов за 2 недели до даты представления. 

#ТеатрГлагол #глаголтеатр #ПолитехПетра #СПбПУ #glagoltheater #theaterglagol #ТеатрыПетербурга #ТеатрыСПб
------------------------------
Пост #2:
15 ноября в Народном университетском театре «Глагол»
Санкт